In [1]:
import arbor
from arbor import mpoint
from arbor import mnpos

In [2]:
tree = arbor.segment_tree()
# Start with a cylinder segment for the soma (with tag 1)
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint( 4,  0.0, 0, 2.0), tag=1)
# Construct the first section of the dendritic tree,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(4,   0.0, 0, 0.8), mpoint( 8,  0.0, 0, 0.8), tag=3)
tree.append(1,     mpoint(8,   0.0, 0, 0.8), mpoint(12, -0.5, 0, 0.8), tag=3)
# Construct the rest of the dendritic tree.
tree.append(2,     mpoint(12, -0.5, 0, 0.8), mpoint(20,  4.0, 0, 0.4), tag=3)
tree.append(3,     mpoint(20,  4.0, 0, 0.4), mpoint(26,  6.0, 0, 0.2), tag=3)
tree.append(2,     mpoint(12, -0.5, 0, 0.5), mpoint(19, -3.0, 0, 0.5), tag=3)
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(24, -7.0, 0, 0.2), tag=3)
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(23, -1.0, 0, 0.2), tag=3)
tree.append(7,     mpoint(23, -1.0, 0, 0.2), mpoint(26, -2.0, 0, 0.2), tag=3)
# Two segments that define the axon, with the first at the root, where its proximal
# end will be connected with the proximal end of the soma segment.
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint(-7,  0.0, 0, 0.4), tag=2)
tree.append(9,     mpoint(-7,  0.0, 0, 0.4), mpoint(-10, 0.0, 0, 0.4), tag=2)
#
# Get the morphology
morph = arbor.morphology(tree)
#
# Create the cable cell
labels = arbor.label_dict( {"soma": "(tag 1)", "axon": "(tag 2)", "dend": "(tag 3)",
                            "stim": "(location 0 0.5)", "everywhere": "(all)"} )
cell = arbor.cable_cell(tree, labels)

# Objectives:
1. **Set global properties**
2. **Set channel mechanisms**
3. **Attach the mechanism**
4. **Set stimulus**
5. **Adding spike detector**
6. **Compartmentalizing the cell**

# 1. Set global cell properties
<img src="images/Arbor_CellSetup_global.png" width="40%" height="40%">

In [3]:
cell.set_properties(Vm=-70, cm=1.0, rL=500.) # tempK=30+273.5
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)

# 2. Set channel mechanisms
<img src="images/generic_expression_for_Conductance.svg" title="http://latex.codecogs.com/svg.latex?G_x = g_x m^p h^q" />

such that,

| _x_  | _p_   | _q_ |
| :---: | :----: | :--: |
|  Na  | 3      | 1   |
|  K    | 4      | 0   |
|  lk   | 0      | 0   |

[NB: <i>m</i><sup>4</sup> = <i>n</i><sup>4</sup>]

Notation in Arbor
*  <i>G<sub>Na</sub></i> &rarr; `gna`
*  <i>G<sub>K</sub></i> &rarr; `gk`
*  <i>G<sub>lk</sub></i> &rarr; `gl` (&rarr; `g` in `passive` mechanism)
*  <i>g<sub>Na</sub></i> &rarr; `gnabar`
*  <i>g<sub>K</sub></i> &rarr; `gkbar`

In [4]:
leaky_chnl = arbor.mechanism('pas', {'e': -54.3, 'g': 0.0003})
hh_na_k_chnl = arbor.mechanism("hh", {"gnabar": 0.120,
                                      "gkbar": 0.036,
                                      "gl": 0.}) # to avoid duplication

# 3. Attaching the mechanism
<img src="images/Arbor_CellSetup_decorate.png" width="40%" height="40%">

In Arbor attaching to a cell is referred to as [decoration](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#decoration). One can decorate a cell by _painting_ or _placing_.

| Painting                    | Placing                       |
|:----------------------------|:------------------------------|
| to regions of a cell to set | to locations on a cell to set |
| * Cable properties          | * Synapses                    |
| * Density mechanisms        | * Gap junction sites          |
| * Ion species               | * Threshold (spike) detectors |
|                             | * Stimuli                     |
|                             | * Probes                      |

For our purpose of attaching `arbor.mechanism` one must therefore paint.

## 3.1. Set reversal potential and attach mechanism to soma

In [5]:
cell.paint('"soma"', "na", rev_pot=50)
cell.paint('"soma"', "k", rev_pot=-77)
cell.paint('"soma"', hh_na_k_chnl)

## 3.2. Similarly for axon and dendrite

In [6]:
cell.paint('"axon"', "na", rev_pot=50)
cell.paint('"axon"', "k", rev_pot=-77)
cell.paint('"axon"', hh_na_k_chnl)
#
cell.paint('"dend"', "na", rev_pot=50)
cell.paint('"dend"', "k", rev_pot=-77)
cell.paint('"dend"', hh_na_k_chnl)

Note that all the above attaching can be done as
```
cell.paint('(join (region "soma") (region "dend") (region "axon"))', "na", rev_pot=50)
cell.paint('(join (region "soma") (region "dend") (region "axon"))', "k", rev_pot=-77)
cell.paint('(join (region "soma") (region "dend") (region "axon"))', hh_na_k_chnl)
```

## 3.3. Attaching a mechanism to all the regions of the cell

The resolution options of _painting_ are shown below

| to paint               | region | cell | global |
|:-----------------------|:------:|:----:|:------:|
| cable properties       | ✓      |  ✓  |  ✓     |
| ion initial conditions | ✓      |  ✓  |  ✓     |
| density mechnism       | ✓      | -   | -       |
| ion rev pot mechanism  | -      |  ✓  | ✓      |
| ion valence            | -      | -   |  ✓     |

In [7]:
cell.paint('"everywhere"', leaky_chnl) # instead for looping through each segment

# *Side-note*
## *1.1 Set cable properties for a particular region*

In [8]:
cell.paint('"soma"', rL=100)
cell.paint("(tag 2)", cm=0.05, rL=80)

# 4. Set stimulus

<pre>
         ___ 0.8nA            ___ 0.8nA         ___ 0.8nA
        |   |                |   |             |   |
________|   |________________|   |_____________|   |______
       / 2ms                 / 2ms             / 2ms
   10ms                  50ms              80ms
</pre>

In [9]:
cell.place('"stim"', arbor.iclamp(10, 2, 0.8))
cell.place('"stim"', arbor.iclamp(50, 2, 0.8))
cell.place('"stim"', arbor.iclamp(80, 2, 0.8))

# 5. Spike detectors with threshold

In [10]:
cell.place('"stim"', arbor.spike_detector(-10)) # -10mV threshold

# 6. Compartmentalizing the cell
Although the cell was created painstakingly by growing the tree piece by piece we have "not really" compartmentalized the cell.
- The typical view of compartmentalizing the cell
- The default Arbor compartmentalization
- Overwriting the default

## 6.1. The typical view of compartmentalizing the cell
<img src="images/MultiCompartmentTypical.gif" width="40%" height="40%">

## 6.2. The default Arbor compartmentalization
<img src="images/MultiCompartmentArborDefault.gif" width="40%" height="40%">

## 6.1. Increase precision in the discretization

In [11]:
cell.compartments_length(2) # Compartment every 2um

<img src="images/MultiCompartmentArborDiscrete.gif" width="40%" height="40%">